**Elaboración de Funciones API**

En esta fase, llevamos a cabo la creación y prueba de funciones esenciales para nuestra API, utilizando los datos disponibles. No obstante, al implementarlas finalmente, es posible que se requieran adaptaciones para cumplir con los requisitos específicos de FastAPI y Render.

**Cargamos las librerias**

In [13]:
import pandas as pd
import warnings
warnings.filterwarnings("ignore")
from sklearn.metrics.pairwise import cosine_similarity

**Funcion PlayTimeGenre**

In [2]:
df_PlayTimeGenre = pd.read_csv("../Archivos Json/df_PlayTimeGenre_hour_final1.csv")
df_PlayTimeGenre.columns

Index(['main_genre', 'release_year', 'playtime_hour'], dtype='object')

In [3]:
def PlayTimeGenre( genero : str ):
    """
    Funcion que devuelve el año con mas horas jugadas para dicho género.
    """
    generos = df_PlayTimeGenre[df_PlayTimeGenre["main_genre"]== genero] #Filtramos en el dataframe el genero que fue solicitado
    if generos.empty:  #Con esta linea nos aseguramos que si para ese genero no hay resultado se notifique
        return f"No se encontraron datos para el género {genero}"
    año_max = generos.loc[generos["playtime_hour"].idxmax()] #Primero identificamos la fila (indice) que tiene la máxima cantidad de horas jugadas para el género dado y posteriormente se selecciona esa fila a partir del indice
    result = {
        'Genero': genero,
        'Año con Más Horas Jugadas': int(año_max["release_year"]),
        'Total de Horas Jugadas': año_max["playtime_hour"]
    }

    return result

In [4]:
PlayTimeGenre("Action")

{'Genero': 'Action',
 'Año con Más Horas Jugadas': 2013,
 'Total de Horas Jugadas': 6419735.3}

**Funcion UserForGenre**

In [6]:
df_UserForGenre = pd.read_csv("../Archivos Json\df_UsersForGenre2_final.csv")
df_UserForGenre.columns

Index(['user_id', 'main_genre', 'release_year', 'playtime_hour'], dtype='object')

In [7]:
def UserForGenre( genero : str ):
    """
    Funcion que devuelve el usuario que acumula más horas jugadas para el género dado 
    y una lista de la acumulación de horas jugadas por año.
    """
    generos2 = df_UserForGenre[df_UserForGenre["main_genre"]== genero]
    user_max = generos2.loc[generos2["playtime_hour"].idxmax()]["user_id"]
    horas_x_año = generos2.groupby(["release_year"])["playtime_hour"].sum().reset_index()
    horas_lista = horas_x_año.rename(columns={"release_year": "Año", "playtime_hour": "Horas"}).to_dict(orient="records")    
    result2 = {
        "Genero": genero,
        "Usuario con Más Horas Jugadas": user_max,
        "Total de Horas Jugadas Por Año": horas_lista
    }
    return result2

In [8]:
UserForGenre("RPG")

{'Genero': 'RPG',
 'Usuario con Más Horas Jugadas': '76561198032459795',
 'Total de Horas Jugadas Por Año': [{'Año': 1997, 'Horas': 38.58333333333333},
  {'Año': 1998, 'Horas': 329.53333333333336},
  {'Año': 2002, 'Horas': 12.483333333333333},
  {'Año': 2003, 'Horas': 33.21666666666667},
  {'Año': 2004, 'Horas': 4.516666666666667},
  {'Año': 2005, 'Horas': 68.63333333333334},
  {'Año': 2007, 'Horas': 142.06666666666666},
  {'Año': 2008, 'Horas': 1680.6833333333332},
  {'Año': 2009, 'Horas': 303.5333333333333},
  {'Año': 2010, 'Horas': 1059.6666666666667},
  {'Año': 2011, 'Horas': 142.81666666666666},
  {'Año': 2012, 'Horas': 17676.55},
  {'Año': 2013, 'Horas': 1661.2333333333333},
  {'Año': 2014, 'Horas': 2739.2833333333333},
  {'Año': 2015, 'Horas': 29931.9},
  {'Año': 2016, 'Horas': 27.53333333333333}]}

In [9]:
df_UserForGenre3 = pd.read_csv("../Archivos Json/df_UsersForGenre3_final.csv")
df_UserForGenre3.columns

Index(['user_id', 'main_genre', 'year_posted', 'playtime_hour'], dtype='object')

In [10]:
def UserForGenre3( genero : str ):
    """
    Funcion que devuelve el usuario que acumula más horas jugadas para el género dado 
    y una lista de la acumulación de horas jugadas por año.
    """
    generos3 = df_UserForGenre3[df_UserForGenre3["main_genre"]== genero]
    user_max3 = generos3.loc[generos3["playtime_hour"].idxmax()]["user_id"]
    horas_x_año3 = generos3.groupby(["year_posted"])["playtime_hour"].sum().reset_index()
    horas_lista3 = horas_x_año3.to_dict(orient="records")
    result6 = {
        "Genero": genero,
        "Usuario con Más Horas Jugadas": user_max3,
        "Total de Horas Jugadas Por Año": horas_lista3
    }
    return result6

In [11]:
UserForGenre3("Adventure")

{'Genero': 'Adventure',
 'Usuario con Más Horas Jugadas': '76561197972452208',
 'Total de Horas Jugadas Por Año': [{'year_posted': 2010,
   'playtime_hour': 5.233333333333333},
  {'year_posted': 2011, 'playtime_hour': 16.5},
  {'year_posted': 2012, 'playtime_hour': 463.45},
  {'year_posted': 2013, 'playtime_hour': 18430.966666666667},
  {'year_posted': 2014, 'playtime_hour': 8068.583333333333},
  {'year_posted': 2015, 'playtime_hour': 24429.566666666666}]}

**Funcion UsersRecommend**

In [12]:
df_UsersRecommend = pd.read_csv("../Archivos Json/df_UsersRecommend2_final.csv")
df_UsersRecommend

,year_posted,item_id,app_name,recommendation_count
0,2010,440,Team Fortress 2,10
1,2010,1250,Killing Floor,6
2,2010,630,Alien Swarm,4
3,2011,440,Team Fortress 2,72
4,2011,620,Portal 2,24
5,2011,105600,Terraria,20
6,2012,440,Team Fortress 2,234
7,2012,4000,Garry's Mod,36
8,2012,105600,Terraria,36
9,2013,440,Team Fortress 2,938


In [13]:
def UsersRecommend( año : int ):
    """
    Funcion que devuelve el top 3 de juegos MÁS recomendados por usuarios para el año dado.
    """
    df_año= df_UsersRecommend[df_UsersRecommend["year_posted"]== año]
    if type(año) != int:
        return {"Debes colocar el año en entero, Ejemplo:2012"}
    if año < df_UsersRecommend["year_posted"].min() or año > df_UsersRecommend["year_posted"].max():
        return {"Año no encontrado"}
    df_ordenado_recomendacion = df_año.sort_values(by="recommendation_count", ascending=False)
    top_3_juegos = df_ordenado_recomendacion.head(3)[["app_name","recommendation_count"]]
    result3 ={
        "Año": año,
        "Top 3 Juegos Más Recomendados": top_3_juegos.to_dict(orient="records")
    }
    return result3

In [14]:
UsersRecommend(2015)

{'Año': 2015,
 'Top 3 Juegos Más Recomendados': [{'app_name': 'Counter-Strike: Global Offensive',
   'recommendation_count': 1305},
  {'app_name': 'Team Fortress 2', 'recommendation_count': 582},
  {'app_name': "Garry's Mod", 'recommendation_count': 294}]}

**Funcion UsersWorstDeveloper**

In [15]:
df_UsersWorstDeveloper = pd.read_csv("../Archivos Json/df_UsersWorstDeveloper_final1.csv")
df_UsersWorstDeveloper

,year_posted,developer,recommendation_count
0,2011,Broken Rules,1
1,2011,Ubisoft Montpellier,1
2,2011,Valve,1
3,2012,NeoCoreGames,1
4,2012,Valve,1
5,2012,Volition,1
6,2013,Hello Games,52
7,2013,Valve,57
8,2014,Bohemia Interactive,19
9,2014,"Trek Industries, Inc",14


In [16]:
def UsersWorstDeveloper( año : int ):
    """
    Funcion que devuelve el top 3 de desarrolladoras con juegos MENOS 
    recomendados por usuarios para el año dado.
    """
    df_año2 = df_UsersWorstDeveloper[df_UsersWorstDeveloper["year_posted"]== año]
    if type(año) != int:
        return {"Debes colocar el año en entero, Ejemplo:2012"}
    if año < df_UsersRecommend["year_posted"].min() or año > df_UsersRecommend["year_posted"].max():
        return {"Año no encontrado "}
    df_ordenado_recomendacion2 = df_año2.sort_values(by="recommendation_count", ascending=False)
    top_3_developers = df_ordenado_recomendacion2.head(3)[["developer","recommendation_count"]]
    result4 = {
        'Año': año,
        'Top 3 Desarrolladoras Menos Recomendadas': top_3_developers.to_dict(orient="records")
    }
    return result4

In [17]:
UsersWorstDeveloper(2015)

{'Año': 2015,
 'Top 3 Desarrolladoras Menos Recomendadas': [{'developer': 'Valve',
   'recommendation_count': 45},
  {'developer': 'Bohemia Interactive', 'recommendation_count': 29},
  {'developer': 'Facepunch Studios', 'recommendation_count': 19}]}

**Funciones sentiment_analysis**

In [18]:
df_Sentiment_Analysis = pd.read_csv("../Archivos Json/df_Sentiment_analysis_final.csv")
df_Sentiment_Analysis

,developer,sentiment_analysis,sentiment_analysis_count
0,07th Expansion,0,1
1,07th Expansion,1,2
2,"10th Art Studio,Adventure Productions",0,1
3,"10th Art Studio,Adventure Productions",1,1
4,10tons Ltd,1,1
...,...,...,...
3977,"インレ,Inre",1,4
3978,橘子班,0,1
3979,橘子班,1,2
3980,橘子班,2,1


In [19]:
def sentiment_analysis( desarrolladora : str ):
    """
    Funcion que devuelve un diccionario con el nombre de la desarrolladora como llave y una lista 
    con la cantidad total de registros de reseñas de usuarios que se encuentren categorizados con 
    un análisis de sentimiento como valor.
    """
    if type(desarrolladora) != str:
        return "Debes colocar un developer de tipo str, EJ:'07th Expansion'"
    if len(desarrolladora) == 0:
        return "Debes colocar un developer en tipo String"
    df_developer = df_Sentiment_Analysis[df_Sentiment_Analysis["developer"]== desarrolladora]
    sentiment_counts = df_developer.groupby("sentiment_analysis")["sentiment_analysis_count"].sum().to_dict()
    sentiment_dicc = {0: "Negativo", 1: "Neutral", 2: "Positivo"}
    sentiment_counts = {sentiment_dicc[key]: value for key, value in sentiment_counts.items()}
    result50 = {desarrolladora: sentiment_counts}
    return result50

In [20]:
sentiment_analysis("Valve")

{'Valve': {'Negativo': 1058, 'Neutral': 4070, 'Positivo': 4330}}

**Recomendaciones**

In [16]:
modelo_reco = pd.read_csv("../Datasets\modelo_reco_final.csv")
modelo_reco

,item_id,app_name,user_id,_Action,_Action Adventure,_Action Adventure Casual Free to Play Indie RPG,_Action Adventure Casual Indie,_Action Adventure Casual Indie RPG Strategy,_Action Adventure Casual Indie Strategy Early Access,_Action Adventure Free to Play Massively Multiplayer RPG,...,_RPG Strategy,_Racing Simulation,_Racing Simulation Sports,_Racing Sports,_Simulation,_Simulation Sports,_Simulation Strategy,_Simulation Strategy Early Access,_Sports,_Strategy
0,10,Pool Nation FX Lite,jonasdbomb,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,80,The Aquatic Adventure of the Last Human Delux...,76561198089393905,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,100,Manyland,jonasdbomb,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,240,The Ritual on Weylyn Island,jonasdbomb,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,280,BOROS,sad-commie,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,300,Beach Rules,76561197970982479,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,360,MX vs ATV Supercross Encore Suzuki RMZ MX,76561198089393905,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
7,360,Street Fighter X Tekken Street Fighter Boost G...,POOTISSPENSOR,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,360,Trump Simulator,ABSOLUTLY_NOTHING,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,380,HITMAN Blood Money Requiem Pack,PPanther,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


**Función recomendacion_juego ( id de producto )**

Ingresando el id de producto, deberíamos recibir una lista con 5 juegos recomendados similares al ingresado.

In [17]:
def encontrar_juegos(item_id):
    # Encuentra el índice del juego ingresado por item_id
    juego_indice = modelo_reco.index[modelo_reco['item_id'] == item_id].tolist()[0]
    
    # Extraigo las características del juego 
    juego_caracteristicas = modelo_reco.iloc[juego_indice, 3:].values.reshape(1, -1)
    
    # Y acá se calcula la similitud coseno entre el juego ingresado y todos los demás juegos
    render = cosine_similarity(modelo_reco.iloc[:, 3:], juego_caracteristicas)
    
    # Obtiene los índices de los juegos más similares (excluyendo el juego de entrada)
    lineas_juegos_similares = render.argsort(axis=0)[::-1][1:6]
    lineas_juegos_similares = lineas_juegos_similares.flatten()[1:]
    
    # Obtiene los juegos más similares en función de los índices
    juegos_similares = modelo_reco.iloc[lineas_juegos_similares]['app_name']
    
    return juegos_similares

In [18]:
juegos_similares = encontrar_juegos(8870)
print(juegos_similares)

1    The Aquatic Adventure of the Last Human  Delux...
2                                             Manyland
3                          The Ritual on Weylyn Island
4                                                BOROS
Name: app_name, dtype: object


**Función recomendacion_usuario ( id de usuario)**

Ingresando el id de un usuario, deberíamos recibir una lista con 5 juegos recomendados para dicho usuario.

In [19]:
def recomendacion_usuario(user_id, modelo_reco):
    # Encuentra con el user_id los juegos recomendados
    if user_id in modelo_reco['user_id'].values:
        juegos = modelo_reco.index[modelo_reco['user_id'] == user_id].tolist()[0]
        
        juego_caracteristicas = modelo_reco.iloc[juegos, 3:].values.reshape(1, -1)
        
        render_similitud = cosine_similarity(modelo_reco.iloc[:, 3:], juego_caracteristicas)
        juegos_similaresrecomend = render_similitud.argsort(axis=0)[::-1][1:6]
        juegos_similaresrecomend = juegos_similaresrecomend.flatten()[1:]
        juegos_similares = modelo_reco.iloc[juegos_similaresrecomend]['app_name']
        
        return juegos_similares  
    else:
        return "El juego con el user_id especificado no existe en la base de datos."
    
user_id = 'sad-commie'
juegos_recomendados = recomendacion_usuario(user_id, modelo_reco)

print(juegos_recomendados) 

26                 Ride
32    Mysterious Castle
8      Trump Simulator 
12        Lethal League
Name: app_name, dtype: object
